##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting started with [TensorBoard.dev](https://tensorboard.dev)

[TensorBoard.dev](https://tensorboard.dev) provides a managed [TensorBoard](https://tensorflow.org/tensorboard) experience that lets you upload and share your ML experiment results with everyone.

This notebook trains a simple model and shows how to upload the logs to TensorBoard.dev.

### Setup and imports

In [ ]:
import tensorflow as tf
import datetime

from tensorboard.plugins.hparams import api as hp

### Train a simple model and create TensorBoard logs

Using the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset as the example, normalize the data and write a function that creates a simple Keras model for classifying the images into 10 classes

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

When training with Keras's [Model.fit()](https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#fit), adding the `tf.keras.callbacks.TensorBoard` callback ensures that logs are created and stored. Additionally, enable histogram computation every epoch with `histogram_freq=1` (this is off by default). Also enable simple hyperparameter logging using `hp.KerasCallback` (more information about hyperparameter logging can be found in the [HParams Dashboard colab](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams)).


In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
hparams_callback = hp.KerasCallback(log_dir, {
    'num_relu_units': 512,
    'dropout': 0.2
})

model.fit(x=x_train,
          y=y_train,
          epochs=5,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback, hparams_callback])

### Upload to TensorBoard.dev

Uploading the TensorBoard logs will give a link that can be shared with anyone. Note that uploaded TensorBoards are public. Do not upload sensitive data.

The uploader will keep running until it is stopped, in order to read new data from the directory during ongoing training.

In [ ]:
!tensorboard dev upload --logdir ./logs \
  --name "Simple experiment with MNIST" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb"

Note that each experiment you upload has a unique experiment ID.  Even if you start a new upload with the same directory, you will get a new experiment ID. It is possible to look at the list of experiments you have uploaded using the `list` command.

In [ ]:
!tensorboard dev list

To remove an experiment you have uploaded, you may use the `delete` command and specifiy the appropriate experiment_id.

In [ ]:
# You must replace YOUR_EXPERIMENT_ID with the value output from the previous
# tensorboard `list` command or `upload` command.  For example
# `tensorboard dev delete --experiment_id pQpJNh00RG2Lf1zOe9BrQA`

## !tensorboard dev delete --experiment_id YOUR_EXPERIMENT_ID_HERE